### Reading data

In [4]:

from sclibrary.io.network_reader import read_B1_B2, read_B2, read_flow
import numpy as np

path = "data/lastfm-dataset-1K"
scbuilder = read_B1_B2(f"{path}/B1.csv")
edges = np.asarray(scbuilder.edges)
triangles = read_B2(f"{path}/B2t.csv", edges=edges)
sc = scbuilder.to_simplicial_complex(triangles=triangles)

print("nodes", len(sc.nodes))
print("edges", len(sc.edges))
print("triangles", len(sc.triangles))

nodes 3092
edges 5507
triangles 571
